# 1、keras模块介绍

### Optimizers

顾名思义，Optimizers包含了一些优化的方法，比如最基本的随机梯度下降SGD,另外还有Adagrad、Adadelta、RMSprop、Adam，一些新的方法以后也会被不断添加进来。

keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.9, nesterov=False)

上面的代码是SGD的使用方法，lr表示学习速率,momentum表示动量项，decay是学习速率的衰减系数(每个epoch衰减一次),Nesterov的值是False或者True，表示使不使用Nesterov momentum

### Objectives

这是目标函数模块，keras提供了mean_squared_error，mean_absolute_error ，squared_hinge，hinge，binary_crossentropy，categorical_crossentropy这几种目标函数。

这里binary_crossentropy 和 categorical_crossentropy也就是常说的logloss.


### Activations

这是激活函数模块，keras提供了linear、sigmoid、hard_sigmoid、tanh、softplus、relu、softplus，另外softmax也放在Activations模块里(我觉得放在layers模块里更合理些）。此外，像LeakyReLU和PReLU这种比较新的激活函数，keras在keras.layers.advanced_activations模块里提供。

### Initializations

这是参数初始化模块，在添加layer的时候调用init进行初始化。keras提供了uniform、lecun_uniform、normal、orthogonal、zero、glorot_normal、he_normal这几种。 

### layers

layers模块包含了core、convolutional、recurrent、advanced_activations、normalization、embeddings这几种layer。

其中core里面包含了flatten(CNN的全连接层之前需要把二维特征图flatten成为一维的)、reshape（CNN输入时将一维的向量弄成二维的）、dense(就是隐藏层，dense是稠密的意思),还有其他的就不介绍了。convolutional层基本就是Theano的Convolution2D的封装。


### Preprocessing

这是预处理模块，包括序列数据的处理，文本数据的处理，图像数据的处理。重点看一下图像数据的处理，keras提供了ImageDataGenerator函数,实现data augmentation，数据集扩增，对图像做一些弹性变换，比如水平翻转，垂直翻转，旋转等。

### Models

这是最主要的模块，模型。上面定义了各种基本组件，model是将它们组合起来，下面通过一个实例来说明。

# 2、一个实例：用CNN分类Mnist

### 读取图片数据

keras要求输入的数据格式是numpy.array类型（numpy是一个python的数值计算的库），所以需要写一个脚本来读入mnist图像，保存为一个四维的data，还有一个一维的label，代码：

In [2]:
#coding:utf-8
"""
Author:wepon
Source:https://github.com/wepe
file:data.py
"""

import os
from PIL import Image
import numpy as np

#读取文件夹mnist下的42000张图片，图片为灰度图，所以为1通道，
#如果是将彩色图作为输入,则将1替换为3，并且data[i,:,:,:] = arr改为data[i,:,:,:] = [arr[:,:,0],arr[:,:,1],arr[:,:,2]]
def load_data():
    data = np.empty((42000,1,28,28),dtype="float32") #42000个一通道的数组，数组格式为28*28
    label = np.empty((42000,),dtype="uint8")

    imgs = os.listdir("E:/machine_data/mnist")   #os.listdir获取目录的内容
    num = len(imgs)
    for i in range(num):
        img = Image.open("E:/machine_data/mnist/"+imgs[i])
        arr = np.asarray(img,dtype="float32")  #将输入数据（列表的列表，元组的元组，元组的列表等）转换为矩阵形式
        data[i,:,:,:] = arr
        label[i] = int(imgs[i].split('.')[0])
    return data,label

In [7]:
imgs = os.listdir("E:/machine_data/mnist")
print (imgs[1:10])

['0.1.jpg', '0.10.jpg', '0.100.jpg', '0.1000.jpg', '0.1001.jpg', '0.1002.jpg', '0.1003.jpg', '0.1004.jpg', '0.1005.jpg']


In [8]:
num = len(imgs)
print (num)

42000


In [21]:
img = Image.open("E:/machine_data/mnist/"+imgs[1])
arr = np.asarray(img,dtype="float32")
print (arr[0:1]) #28*28
print (arr.shape)

[[  0.   4.   6.   0.   0.   0.   0.   0.   1.   0.   0.   3.   0.   2.
    0.   3.   0.   2.  11.   6.   0.   0.   3.   1.   0.   0.   0.   0.]]
(28, 28)


In [14]:
data = np.empty((42000,1,28,28),dtype="float32")

In [16]:
data[1,:,:,:] = arr

In [22]:
print (data.shape)
print (data[1,0,:,:][0:1])

(42000, 1, 28, 28)
[[  0.   4.   6.   0.   0.   0.   0.   0.   1.   0.   0.   3.   0.   2.
    0.   3.   0.   2.  11.   6.   0.   0.   3.   1.   0.   0.   0.   0.]]


In [26]:
imgs[10000].split('.')

['2', '2063', 'jpg']

### 构建CNN，训练

In [7]:
#导入各种用到的模块组件
from __future__ import absolute_import
from __future__ import print_function
#from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from six.moves import range

from keras import backend as K
K.set_image_dim_ordering('th')

#加载数据
data, label = load_data()
print(data.shape[0], ' samples')

#label为0~9共10个类别，keras要求格式为binary class matrices,转化一下，直接调用keras提供的这个函数
label = np_utils.to_categorical(label, 10)
print (label[1:3])

###############
#开始建立CNN模型
###############

#生成一个model
model = Sequential()

#第一个卷积层，4个卷积核，每个卷积核大小5*5。1表示输入的图片的通道,灰度图为1通道。
#border_mode可以是valid或者full，具体看这里说明：http://deeplearning.net/software/theano/library/tensor/nnet/conv.html#theano.tensor.nnet.conv.conv2d
#激活函数用tanh
#你还可以在model.add(Activation('tanh'))后加上dropout的技巧: model.add(Dropout(0.5))
#input_shape=(3,224,224) is for "theano",   "tensorflow" should be input_shape=(224,224,3)
model.add(Convolution2D(4,5,5,border_mode='valid', input_shape=(1,28,28))) 
model.add(Activation('tanh'))

#第二个卷积层，8个卷积核，每个卷积核大小3*3。4表示输入的特征图个数，等于上一层的卷积核个数
#激活函数用tanh
#采用maxpooling，poolsize为(2,2)
model.add(Convolution2D(8,3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#第三个卷积层，16个卷积核，每个卷积核大小3*3
#激活函数用tanh
#采用maxpooling，poolsize为(2,2)
model.add(Convolution2D(16,3, 3, border_mode='valid')) 
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#全连接层，先将前一层输出的二维特征图flatten为一维的。
#Dense就是隐藏层。16就是上一层输出的特征图个数。4是根据每个卷积层计算出来的：(28-5+1)得到24,(24-3+1)/2得到11，(11-3+1)/2得到4
#全连接有128个神经元节点,初始化方式为normal
model.add(Flatten())
model.add(Dense(16*4*4))
model.add(Activation('tanh'))

#Softmax分类，输出是10类别
model.add(Dense(10))
model.add(Activation('softmax'))

#############
#开始训练模型
##############
#使用SGD + momentum
#model.compile里的参数loss就是损失函数(目标函数)
sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,class_mode="categorical",metrics=['accuracy'])

#调用fit方法，就是一个训练过程. 训练的epoch数设为10，batch_size为100．
#数据经过随机打乱shuffle=True。verbose=1，训练过程中输出的信息，0、1、2三种方式都可以，无关紧要。show_accuracy=True，训练时每一个epoch都输出accuracy。
#validation_split=0.2，将20%的数据作为验证集。
model.fit(data, label, batch_size=100,nb_epoch=3,shuffle=True,verbose=1,validation_split=0.2)

42000  samples
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


D:\anaconda1\lib\site-packages\ipykernel\__main__.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (5, 5), input_shape=(1, 28, 28..., padding="valid")`
D:\anaconda1\lib\site-packages\ipykernel\__main__.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="valid")`
D:\anaconda1\lib\site-packages\ipykernel\__main__.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="valid")`
D:\anaconda1\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
D:\anaconda1\lib\site-packages\keras\backend\tensorflow_backend.py:2250: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 33600 samples, validate on 8400 samples
Epoch 1/3
33600/33600 [==============================] - 29s - loss: 0.1723 - acc: 0.9460 - val_loss: 8.5907 - val_acc: 0.0173
Epoch 2/3
33600/33600 [==============================] - 29s - loss: 0.0775 - acc: 0.9752 - val_loss: 9.5697 - val_acc: 0.0170
Epoch 3/3
33600/33600 [==============================] - 28s - loss: 0.0572 - acc: 0.9813 - val_loss: 10.4809 - val_acc: 0.0174


In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils

from keras import backend as K
K.set_image_dim_ordering('th')

#加载数据
data, label = load_data()
print(data.shape[0], ' samples')

#label为0~9共10个类别，keras要求格式为binary class matrices,转化一下，直接调用keras提供的这个函数
label = np_utils.to_categorical(label, 10)
print (label[1:3])
X_train=data
Y_train=label

model = Sequential()
# input: 100x100 images with 3 channels -> (3, 100, 100) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution2D(4, 5, 5, border_mode='valid', input_shape=(1, 28, 28)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(8, 3, 3))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Convolution2D(64, 3, 3, border_mode='valid'))
# model.add(Activation('relu'))
# model.add(Convolution2D(64, 3, 3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
# Note: Keras does automatic shape inference.
model.add(Dense(100))
model.add(Activation('tanh'))

model.add(Dense(10))
model.add(Activation('softmax'))

sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=100, nb_epoch=3,validation_split=0.2)

42000  samples
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


D:\anaconda1\lib\site-packages\ipykernel\__main__.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (5, 5), input_shape=(1, 28, 28..., padding="valid")`
D:\anaconda1\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3))`
D:\anaconda1\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 33600 samples, validate on 8400 samples
Epoch 1/3
33600/33600 [==============================] - 15s - loss: 0.3226 - acc: 0.8961 - val_loss: 8.9395 - val_acc: 0.0169
Epoch 2/3
33600/33600 [==============================] - 14s - loss: 0.1764 - acc: 0.9440 - val_loss: 8.6072 - val_acc: 0.0165
Epoch 3/3
33600/33600 [==============================] - 14s - loss: 0.1511 - acc: 0.9517 - val_loss: 9.5022 - val_acc: 0.0160
